# Beyond the Screen - AR Driven Insights

In [34]:
import plotar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import json

## GAPminder Animated Scatter Plot

Now let's animate the data showing how world wars etc. and general development shaped countries by looking at their income per capita, life expectancy, and population as size  - however, we can use one more dimension! children_per_woman are available for the whole time period.


In [ ]:
url = 'https://github.com/UofTCoders/2018-09-10-utoronto/raw/gh-pages/data/world-data-gapminder.csv'
gap = pd.read_csv(url)
gap

In [ ]:
plot = plotar.animate(gap.query("region=='Europe'"), xyz=['income','children_per_woman','life_expectancy'],
    group='country', col='sub_region', size='population', animation_frame='year',
    label = 'country', name="gapminder-animated-label")
plot

In [ ]:
plot.write("examples/gapminder-animated-label.json", format="json gltf glb usda usdz".split())

## Surface Plot of Switzerland

We use the [Swisstopo Digital Height Model](https://www.swisstopo.admin.ch/de/geodata/height/dhm25200.html) 200m grid to draw a surface of Swizterland.

In [ ]:
url = 'https://data.geo.admin.ch/ch.swisstopo.digitales-hoehenmodell_25/data.zip'
file_name = 'DHM200.asc'

In [ ]:
def get_or_download(url, file_name, cache="tmp"):
    file = Path(cache) / file_name
    if not file.exists():
        from io import BytesIO
        from zipfile import ZipFile
        import shutil
        print(f"Downloading {url} to {file} ...")
        zipfile = ZipFile(BytesIO(requests.get(url).content))
        with open(file, 'wb') as f:
            shutil.copyfileobj(zipfile.open(file_name), f)
        print(f"Downloaded {file} from {url}")
    else:
        print(f"getting {file} from cache")
    return file

In [ ]:
file = get_or_download(url, file_name)

The GeoSpatial Information is in the first 6 rows of the file:

In [ ]:
%%time
y_head = {k: float(v) for k,v in np.genfromtxt(file, dtype=str, max_rows=6)}
print(y_head)
y = np.genfromtxt(file, skip_header=6, skip_footer=1)
y.shape

In [ ]:
n,m = [int(y_head[_]) for _ in ['NCOLS','NROWS']]
n,m

In [ ]:
img = y.flatten()[:n*(m-1)].reshape((m-1,n))

In [ ]:
factor = 2

In [ ]:
factor = 4

In [ ]:
img = img[::factor,::factor]

In [ ]:
xvec = np.arange(img.shape[1]) * y_head['CELLSIZE'] * factor
yvec = np.arange(img.shape[0]) * y_head['CELLSIZE'] * factor

In [ ]:
img[img>0].min()

In [ ]:
img[img<0] = 150

In [ ]:
plt.imshow(img, interpolation='none');

In [ ]:
plot = plotar.surfacevr(img, x=xvec, y=yvec, auto_scale=False, name="CH")
plot

## Surface of Switzerland with color

Now add the official satellite image on top of that surface.

In [ ]:
landsat_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "LandsatMos25.tif")
landsat_metadata_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "Landsatmos25.TFW")

In [ ]:
y_head

In [ ]:
sat_head = np.genfromtxt(landsat_metadata_file).astype(np.int64)
sat_head.T

Description 
* First row is x-pixel resolution
* Second and third rows are so-called "rotational components" but are set to zero in the case of an unrotated mapsheet.
* The fourth row is the y-pixel resolution. The negative sign indicates that the image y-axis is positive down which is the opposite from real world coordinates.
* The 5th and 6th rows are the Easting and Northing of the upper left pixel (0,0 in image coordinates). 

If you compare `y_head` and `sat_head` you see that unfortunately we need to crop the satellite to match the frame of the surface data:

In [ ]:
crop = (
    -sat_head[4] + (y_head['XLLCORNER']),
    sat_head[5] - (y_head['YLLCORNER'] + y_head['CELLSIZE'] * y_head['NROWS']),
)
crop = crop + (
    crop[0] + y_head['CELLSIZE'] * y_head['NCOLS'],
    crop[1] + y_head['CELLSIZE'] * y_head['NROWS'],
)
np.array(crop)/25.0

In [ ]:
from PIL import Image

In [ ]:
landsat = Image.open(landsat_file)

Now crop it and rescale it to the size of the surface

In [ ]:
landsat_small = landsat.crop(np.array(crop)/25.0).resize(reversed(img.shape))

In [ ]:
landsat_small.size, np.array(landsat_small).shape, img.shape

In [ ]:
landsat_small

Now plot it and resize it - also exaggerate the height by a factor ~3

In [ ]:
plot = plotar.surfacevr(img/100000, x=xvec/300000, y=yvec/300000, surfacecolor=np.array(landsat_small).astype(int).tolist(),
                             auto_scale=False, name="CH-color",)

In [ ]:
plot.write("examples/CH-color.json", format="json gltf glb usda usdz".split())